In [ ]:
import math
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import LinearRing

def reward_function(params):
    """
    This is a reward function for the AWS Deep Racer car. The function applies a penalty if the car steers excessively, 
    and also if it does not stay on track. Conversely, it rewards the car for staying on track. If the car is on the 
    left side of the track, it incurs a penalty, but it is rewarded each time it maintains position on the right side of the track. 
    The function uses waypoints and coordinates to calculate a distance. When the car is near these waypoints, it receives rewards; 
    otherwise, it is penalized. The car is rewarded if it stays in the centre of the track, taking the track's width into consideration.
    With the closest waypoints, available as the previous point and the next point to the car, a vector is calculated. 
    This results in an angle, allowing the car to anticipate a curve. If there is no curve, the car is rewarded for increasing speed. 
    Conversely, slowing down when a curve is anticipated is also rewarded.
    """
    
    
    max_speed = 3.5  # maximum speed (used for straight portions of the track)
    min_speed = 1.0  # minimum speed (used for turns)
    abs_steering_threshold = 15 # set a limit for the car to steer when turning 
    track_curve_threshold = math.radians(15) # this boundary will be useful when combined with the car speed
    
    # call the input paramaters
    # position variable is using Shapely to take each point in the track, by its coordinates
    # track variable is making a ring around each waypoint
    # steering variable is set with absolute value
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    position = Point(params['x'], params['y'])
    track = LinearRing(params['waypoints'])
    steering = abs(params['steering_angle'])
    speed = params['speed']
    on_track = params['all_wheels_on_track']
    on_left = params['is_left_of_center']
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    
    # initializing a low reward
    reward = 1e-3

    # adjustment in one example in the input parameters page, with lower steering treshold  
    if steering > abs_steering_threshold:
        reward *= 0.8

    # car is heavy punished when it does not stays on track
    if not on_track:
        reward = 1e-3
    else:
        reward += 0.1
    
    # each time the car stays on the correct lane, it is rewarded
    if on_left:
        reward *= 0.8
    else:
        reward += 0.1

    # with the position and track parameters a point is measured (result is a float)
    point = position.distance(track)
    # if the car goes far from the point, receives a penalty, but reward increases if closer 
    if point < 1:
        reward += 1.2
    elif point < 3:
        reward += 0.9
    elif point < 6:
        reward += 0.6
    elif point < 9:
        reward += 0.3   
    else:
        reward *= 0.3

    # the distance from center example enhanced
    marker_rewards = [('marker_value', float), ('marker_reward', float)]

    # Create the structured array with the marker values
    markers = np.array([
        (0.05 * track_width, 2.0),
        (0.10 * track_width, 1.6),
        (0.15 * track_width, 1.4),
        (0.20 * track_width, 1.2),
        (0.25 * track_width, 0.8),
        (0.30 * track_width, 0.6),
        (0.35 * track_width, 0.4),
        (0.40 * track_width, 0.2),
        (0.50 * track_width, 0.1)
    ], dtype=marker_rewards)

    # iterates over each value to receive rewards, or penalties, based on the distance from center and track width
    for marker in markers:
        marker_value, marker_reward = marker
        if distance_from_center <= marker_value:
            reward *= marker_reward
            break

    # set the closest waypoints        
    current_waypoint = waypoints[closest_waypoints[0]] # index 0 for closest
    next_waypoint = waypoints[closest_waypoints[1]] # index 1 for next closest
    next_next_waypoint_index = (closest_waypoints[1] + 1) % len(waypoints) # % len(waypoints) is used to avoid index errors.
    next_next_waypoint = waypoints[next_next_waypoint_index]

    # subtract x and y coordinate with the closest waypoints to preview the oncoming track 
    direction_vector = [next_waypoint[0] - current_waypoint[0], next_waypoint[1] - current_waypoint[1]]
    next_direction_vector = [next_next_waypoint[0] - next_waypoint[0], next_next_waypoint[1] - next_waypoint[1]]

    # calculate the angle using the direction vectors
    track_angle = math.atan2(next_direction_vector[1], next_direction_vector[0]) - math.atan2(direction_vector[1], direction_vector[0])
    track_angle = abs(track_angle)

    # if there is no curve ahead increase the speed, reduce speed if curve is coming
    if track_angle < track_curve_threshold:
        # reward for max speed on straight tracks
        if speed >= max_speed:
            reward *= 1.15
    else:
        # reward for slower speed on turns/curves
        if speed <= min_speed:
            reward *= 1.10

    # win reward points, or not
    return float(reward)